In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import pickle

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/06/30 08:31:20 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/MLOps-Zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1719736280654, experiment_id='1', last_update_time=1719736280654, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

Link to download the data: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [4]:
def read_datafram(filname):
    df = pd.read_parquet(filname)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
train_df = read_datafram('data/green_tripdata_2021-01.parquet')
val_df = read_datafram('data/green_tripdata_2021-02.parquet')

In [6]:
train_df['PU_DO'] = train_df['PULocationID'] + '_' + train_df['DOLocationID']
val_df['PU_DO'] = val_df['PULocationID'] + '_' + val_df['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = train_df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
y_train = train_df['duration'].values

val_dicts = val_df[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = val_df['duration'].values

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)

print('MSE: ', mse)

MSE:  7.758715209663881


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
with mlflow.start_run():
  
  mlflow.set_tag('developer', 'esmail')
  
  mlflow.log_param('train-data-path', 'data/green_tripdata_2021-01.parquet')
  mlflow.log_param('test-data-path', 'data/green_tripdata_2021-02.parquet')
  
  alpha = 0.01
  mlflow.log_param('alpha', alpha)
  
  ls = Lasso(alpha=alpha)
  ls.fit(X_train, y_train)

  y_pred = ls.predict(X_val)

  rmse = mean_squared_error(y_val, y_pred, squared=False)

  print('RMSE: ', rmse)
  mlflow.log_metric('RMSE', rmse)

RMSE:  11.167275941179728


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [11]:
re = Ridge(alpha=1)
re.fit(X_train, y_train)

y_pred = re.predict(X_val)

mse = mean_squared_error(y_val, y_pred, squared=False)

print('MSE: ', mse)

MSE:  7.703735141467915


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [115]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, re), f_out)